## 4.7: Deriving New Variables

In [304]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [305]:
##Importing path, and csv files of products

path= r'/Users/erictam/Downloads/CareerFoundry Program/Data Immersion Achievement 4/Instacart Basket Analysis'

# To import 4.6 orders_prod_merged.pkl as ords_prods_merge

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', '4.6 orders_prod_merged.pkl'))
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices
0,2539329,1,1,2,8,11.114836,196,1,0,Soda,77,7,9.0
1,2539329,1,1,2,8,11.114836,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5
2,2539329,1,1,2,8,11.114836,12427,3,0,Original Beef Jerky,23,19,4.4
3,2539329,1,1,2,8,11.114836,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7
4,2539329,1,1,2,8,11.114836,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0


In [306]:
## Creating a subset of ords_prods_merge for the first 1 million rows

df= ords_prods_merge[:1000000]

In [307]:
## Defining price_label if statement function

def price_label(row):

  if row['prices'] <= 5:
    return 'Low-range product'
  elif (row['prices'] > 5) and (row['prices'] <= 15):
    return 'Mid-range product'
  elif row['prices'] > 15:
    return 'High range'
  else: return 'Not enough data'


In [308]:
# Applying the price_label function

df['price_range']=df.apply(price_label,axis=1)

/var/folders/c3/gxzbvq9x1032lnrpvbyhdtrw0000gn/T/ipykernel_9272/3807223418.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range']=df.apply(price_label,axis=1)


In [309]:
# Running frequency counts of the new column data

df['price_range'].value_counts(dropna= False)

price_range
Mid-range product    673471
Low-range product    314116
High range            12413
Name: count, dtype: int64

## Fixing An Error in the Data

In [311]:


df['prices'].max()

#There seems to be an error in the data. Going back to the original data provided in achievement 4.3, I was checked that data and it looks like a similar value of 99999 is found throughout the data. This should have been found within the data consistency checking and deleted. Upon filtering for this data, it seems anything > $26 provides orders of ridiculous values like $14,900 for cottage cheese. This most likeley is bad data and makes up 1% of the 1 million rows. This will be deleted going forward. 

99999.0

In [312]:
#Indexing a new df to identify the high range products

df_high_range=df[df['price_range'] == 'High range']


In [313]:
##Describe function shows the data where a max price of the data is $99999. This does NOT seem likely

df_high_range.describe()

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,prices
count,1.241300e+04,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000,12413.000000
mean,1.701999e+06,3212.811971,15.264964,2.720374,13.465157,11.707698,23061.511883,8.675421,0.537582,79.797068,12.050995,311.573181
std,9.874408e+05,1869.108799,16.043024,2.154155,4.103285,8.656993,14676.421917,7.208707,0.498606,38.767033,0.447423,3811.573198
min,2.800000e+02,6.000000,1.000000,0.000000,0.000000,0.000000,35.000000,1.000000,0.000000,7.000000,12.000000,15.100000
25%,8.376450e+05,1608.000000,4.000000,1.000000,10.000000,5.000000,9339.000000,3.000000,0.000000,35.000000,12.000000,18.600000
50%,1.697764e+06,3148.000000,10.000000,2.000000,13.000000,9.000000,20518.000000,7.000000,1.000000,106.000000,12.000000,20.700000
75%,2.558852e+06,4847.000000,21.000000,5.000000,16.000000,15.000000,38190.000000,12.000000,1.000000,106.000000,12.000000,23.100000
max,3.420523e+06,6467.000000,99.000000,6.000000,23.000000,30.000000,49670.000000,76.000000,1.000000,122.000000,16.000000,99999.000000


In [314]:
#Using the describe function, the 75% quartile of prices are $24.7. When filtering up to $26, that is when the ridiculous prices begin. This data will be deleted going forward to make sure further analysis goes smoothly 

df_high_range[df_high_range['prices']>26]

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range
1576,912404,17,12,2,14,5.0,21553,5,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range
1638,603376,17,22,6,16,4.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range
16532,3264360,135,2,2,21,13.0,21553,6,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range
16538,892534,135,3,0,8,12.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range
53710,229704,342,8,1,19,30.0,21553,9,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959878,840159,6217,13,1,11,15.0,21553,4,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range
959924,1235750,6217,20,5,9,11.0,21553,6,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range
961178,40712,6221,5,0,15,8.0,21553,8,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range
961252,2567369,6221,10,2,17,17.0,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,High range


## Editing ords_prods_merge Data frame to delete price rows > $26

In [316]:
# Subsetting and creating a new df

ords_prods = ords_prods_merge[ords_prods_merge['prices'] <= 26]
ords_prods.shape

(32429085, 13)

In [317]:
ords_prods.describe()

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,prices
count,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07,3.242908e+07
mean,1.710748e+06,1.029369e+05,1.714222e+01,2.738808e+00,1.342499e+01,1.110473e+01,2.557815e+04,8.351234e+00,5.896850e-01,7.121119e+01,9.921270e+00,7.795704e+00
std,9.873002e+05,5.946661e+04,1.753501e+01,2.090060e+00,4.246368e+00,8.493005e+00,1.409712e+04,7.126727e+00,4.918909e-01,3.820485e+01,6.281221e+00,4.243149e+00
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,8.559440e+05,5.141600e+04,5.000000e+00,1.000000e+00,1.000000e+01,5.000000e+00,1.353500e+04,3.000000e+00,0.000000e+00,3.100000e+01,4.000000e+00,4.200000e+00
50%,1.711051e+06,1.026110e+05,1.100000e+01,3.000000e+00,1.300000e+01,8.000000e+00,2.526500e+04,6.000000e+00,1.000000e+00,8.300000e+01,9.000000e+00,7.400000e+00
75%,2.565509e+06,1.543900e+05,2.400000e+01,5.000000e+00,1.600000e+01,1.400000e+01,3.794000e+04,1.100000e+01,1.000000e+00,1.070000e+02,1.600000e+01,1.130000e+01
max,3.421083e+06,2.062090e+05,9.900000e+01,6.000000e+00,2.300000e+01,3.000000e+01,4.968800e+04,1.450000e+02,1.000000e+00,1.340000e+02,2.100000e+01,2.500000e+01


## Editing df to reflect a prices max of $14.8 to make it consistent with the Achievement instructions

In [319]:
# Subsetting df (1 million rows) to df_new for < $14.8
df_new= df[df['prices']<=14.8]
df_new.describe()

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id,prices
count,9.798340e+05,979834.000000,979834.000000,979834.000000,979834.000000,979834.000000,979834.000000,979834.000000,979834.000000,979834.000000,979834.000000,979834.000000
mean,1.712126e+06,3292.575839,17.334006,2.765387,13.414256,11.057327,25639.509993,8.303329,0.592171,71.020414,9.968626,7.570545
std,9.869876e+05,1865.049358,17.943301,2.090148,4.222180,8.525029,14079.245802,7.208114,0.491431,38.204893,6.330913,3.964329
min,8.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
25%,8.621382e+05,1698.000000,5.000000,1.000000,10.000000,5.000000,13629.000000,3.000000,0.000000,31.000000,4.000000,4.200000
50%,1.711412e+06,3350.000000,11.000000,3.000000,13.000000,8.000000,25466.000000,6.000000,1.000000,83.000000,8.000000,7.300000
75%,2.563542e+06,4934.000000,24.000000,5.000000,16.000000,14.000000,37886.000000,11.000000,1.000000,107.000000,16.000000,11.000000
max,3.420853e+06,6467.000000,99.000000,6.000000,23.000000,30.000000,49687.000000,95.000000,1.000000,134.000000,21.000000,14.800000


## Returning back to normal achievement instructions

In [321]:
# Applying the price_label function

df_new['price_range']=df_new.apply(price_label,axis=1)
df_new.head()

/var/folders/c3/gxzbvq9x1032lnrpvbyhdtrw0000gn/T/ipykernel_9272/2384442314.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['price_range']=df_new.apply(price_label,axis=1)


,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range
0,2539329,1,1,2,8,11.114836,196,1,0,Soda,77,7,9.0,Mid-range product
1,2539329,1,1,2,8,11.114836,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,Mid-range product
2,2539329,1,1,2,8,11.114836,12427,3,0,Original Beef Jerky,23,19,4.4,Low-range product
3,2539329,1,1,2,8,11.114836,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,Low-range product
4,2539329,1,1,2,8,11.114836,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low-range product


In [322]:
# Running frequency counts of the new column data

df_new['price_range'].value_counts(dropna= False)

#Now, like the achivement instructions, there are no High Range Values. There is a max() of $14.8

price_range
Mid-range product    665718
Low-range product    314116
Name: count, dtype: int64

## If-statements with the loc() function

In [324]:
df_new.loc[df_new['prices'] > 15, 'price_range_loc'] = 'High-range product'

/var/folders/c3/gxzbvq9x1032lnrpvbyhdtrw0000gn/T/ipykernel_9272/3900876137.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.loc[df_new['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [325]:
df_new.loc[(df_new['prices'] <= 15) & (df['prices'] > 5), 'price_range_loc'] = 'Mid-range product' 

In [326]:
df_new.loc[df_new['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [327]:
df_new['price_range_loc'].value_counts(dropna= False)

## The Same counts are found when comparing to the counts through the user-defined if statements

price_range_loc
Mid-range product    665718
Low-range product    314116
Name: count, dtype: int64

In [328]:
#Now the same loc() function will be applied to the ords_prods dataframe

ords_prods.loc[ords_prods['prices'] > 15, 'price_range_loc'] = 'High-range product'

/var/folders/c3/gxzbvq9x1032lnrpvbyhdtrw0000gn/T/ipykernel_9272/936528281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ords_prods.loc[ords_prods['prices'] > 15, 'price_range_loc'] = 'High-range product'


In [329]:
ords_prods.loc[(ords_prods['prices'] <= 15) & (ords_prods['prices'] > 5), 'price_range_loc'] = 'Mid-range product'

In [330]:
ords_prods.loc[ords_prods['prices'] <= 5, 'price_range_loc'] = 'Low-range product'

In [331]:
ords_prods['price_range_loc'].value_counts(dropna= False)

price_range_loc
Mid-range product     21890146
Low-range product     10126384
High-range product      412555
Name: count, dtype: int64

## If-statements with For-Loops

In [333]:
# Value counts to see which day of the week is the most popular
# 0 means saturday per project brief. Saturday is the busiest day for orders. Wednesday is the slowest day for orders

ords_prods['orders_day_of_week'].value_counts(dropna= False)

orders_day_of_week
0    6208779
1    5664898
6    4499643
2    4217041
5    4208734
3    3843427
4    3786563
Name: count, dtype: int64

In [334]:
# For statement for busiest day

result = []

for value in ords_prods["orders_day_of_week"]:
        if value==0:
            result.append('Busiest Day')
        elif value==4:
            result.append('Least Busy')
        else:
            result.append('Regularly Busy')

result

['Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy',
 'Least Busy

In [335]:
#Combining result list to ords_prods dataframe

ords_prods['Busiest_day']=result
ords_prods['Busiest_day'].value_counts(dropna=False)

/var/folders/c3/gxzbvq9x1032lnrpvbyhdtrw0000gn/T/ipykernel_9272/3087704046.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ords_prods['Busiest_day']=result


Busiest_day
Regularly Busy    22433743
Busiest Day        6208779
Least Busy         3786563
Name: count, dtype: int64

In [336]:
ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,Busiest_day
0,2539329,1,1,2,8,11.114836,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly Busy
1,2539329,1,1,2,8,11.114836,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,Mid-range product,Regularly Busy
2,2539329,1,1,2,8,11.114836,12427,3,0,Original Beef Jerky,23,19,4.4,Low-range product,Regularly Busy
3,2539329,1,1,2,8,11.114836,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,Low-range product,Regularly Busy
4,2539329,1,1,2,8,11.114836,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low-range product,Regularly Busy


## Exercise 4.7 Questions Section

In [338]:
# 2. Suppose your clients have changed their minds about the labels you created in your “busiest_day” column. Now, they want “Busiest day” to become “Busiest days” (plural). This label should correspond with the two busiest days of the week as opposed to the single busiest day. At the same time, they’d also like to know the two slowest days. Create a new column for this using a suitable method.

# The TWO busiest days of the week are 0 & 1, or Saturday and Sunday
# The TWO slowest days of the week are 3 & 4, or Tuesday and Wednesday

# This will use a for statement to reflect the necessary changes

# For statement for busiest day

days = []

for value in ords_prods["orders_day_of_week"]:
        if value==0 or value== 1:
            days.append('Busiest days')
        elif value==3 or value == 4:
            days.append('Slowest days')
        else:
            days.append('Regularly Busy')

days


['Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Regularly Busy',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Busiest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest days',
 'Slowest day

In [339]:
# Setting Busiest_day column to equal days variable
# Frequency check to make sure no Nan values

ords_prods['Busiest_day']=days
ords_prods['Busiest_day'].value_counts(dropna=False)

/var/folders/c3/gxzbvq9x1032lnrpvbyhdtrw0000gn/T/ipykernel_9272/3912593787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ords_prods['Busiest_day']=days


Busiest_day
Regularly Busy    12925418
Busiest days      11873677
Slowest days       7629990
Name: count, dtype: int64

In [340]:
ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,Busiest_day
0,2539329,1,1,2,8,11.114836,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly Busy
1,2539329,1,1,2,8,11.114836,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,Mid-range product,Regularly Busy
2,2539329,1,1,2,8,11.114836,12427,3,0,Original Beef Jerky,23,19,4.4,Low-range product,Regularly Busy
3,2539329,1,1,2,8,11.114836,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,Low-range product,Regularly Busy
4,2539329,1,1,2,8,11.114836,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low-range product,Regularly Busy


## #3. Check the values of this new column for accuracy. Note any observations in markdown format.

In [342]:
# 3. Check the values of this new column for accuracy. Note any observations in markdown format.

#Filter dataframe for orders_day of week = 3 as it should say slowest days
ords_prods[ords_prods['orders_day_of_week']==3]


,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,Busiest_day
5,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days
6,2398795,1,2,3,7,15.0,10258,2,0,Pistachios,117,19,3.0,Low-range product,Slowest days
7,2398795,1,2,3,7,15.0,12427,3,1,Original Beef Jerky,23,19,4.4,Low-range product,Slowest days
8,2398795,1,2,3,7,15.0,13176,4,0,Bag of Organic Bananas,24,4,10.3,Mid-range product,Slowest days
9,2398795,1,2,3,7,15.0,26088,5,1,Aged White Cheddar Popcorn,23,19,4.7,Low-range product,Slowest days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434158,2129269,206209,8,3,17,22.0,12376,8,0,Sourdough Demi Baguette,112,3,11.5,Mid-range product,Slowest days
32434159,2129269,206209,8,3,17,22.0,40992,9,1,Chopped Tomatoes,81,15,1.8,Low-range product,Slowest days
32434160,2129269,206209,8,3,17,22.0,33129,10,0,Classic Hummus,67,20,1.3,Low-range product,Slowest days
32434161,2129269,206209,8,3,17,22.0,14870,11,0,Icelandic Style Fat Free Plain Yogurt,120,16,2.1,Low-range product,Slowest days


In [343]:
#Filter dataframe for orders_day of week = 4 as it should say slowest days
ords_prods[ords_prods['orders_day_of_week']==4]

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,Busiest_day
16,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,Mid-range product,Slowest days
17,2254736,1,4,4,7,29.0,12427,2,1,Original Beef Jerky,23,19,4.4,Low-range product,Slowest days
18,2254736,1,4,4,7,29.0,10258,3,1,Pistachios,117,19,3.0,Low-range product,Slowest days
19,2254736,1,4,4,7,29.0,25133,4,1,Organic String Cheese,21,16,8.6,Mid-range product,Slowest days
20,2254736,1,4,4,7,29.0,26405,5,1,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low-range product,Slowest days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434178,1854736,206209,11,4,10,30.0,24852,4,1,Banana,24,4,12.3,Mid-range product,Slowest days
32434179,1854736,206209,11,4,10,30.0,19348,5,1,Fat Free Milk,84,16,4.6,Low-range product,Slowest days
32434180,1854736,206209,11,4,10,30.0,15700,6,1,Cheddar Cheese Nut Thins Cracker Snacks,78,19,2.3,Low-range product,Slowest days
32434181,1854736,206209,11,4,10,30.0,40310,7,0,Oven Roasted Turkey Breast,96,20,1.9,Low-range product,Slowest days


In [344]:
#Filter dataframe for orders_day of week = 0 as it should say Busiest days
ords_prods[ords_prods['orders_day_of_week']==0]

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,Busiest_day
284,2710558,3,5,0,17,12.0,39190,1,1,Vanilla Unsweetened Almond Milk,91,16,12.3,Mid-range product,Busiest days
285,2710558,3,5,0,17,12.0,9387,2,1,Granny Smith Apples,24,4,8.8,Mid-range product,Busiest days
286,2710558,3,5,0,17,12.0,17668,3,1,Unsweetened Chocolate Almond Breeze Almond Milk,91,16,6.6,Mid-range product,Busiest days
287,2710558,3,5,0,17,12.0,47766,4,1,Organic Avocado,24,4,6.3,Mid-range product,Busiest days
288,2710558,3,5,0,17,12.0,16965,5,1,Chocolate Ice Cream,37,1,12.5,Mid-range product,Busiest days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32433917,2808240,206208,38,0,15,19.0,14467,6,1,Sweet Baguette,112,3,4.5,Low-range product,Busiest days
32433918,2808240,206208,38,0,15,19.0,18770,7,1,Uncured Italian Dry Salami,96,20,7.8,Mid-range product,Busiest days
32433919,2808240,206208,38,0,15,19.0,21137,8,1,Organic Strawberries,24,4,7.3,Mid-range product,Busiest days
32434139,3186442,206209,6,0,16,3.0,14197,1,0,Tomato Paste,9,9,5.6,Mid-range product,Busiest days


In [345]:
#Filter dataframe for orders_day of week = 1 as it should say Busiest days
ords_prods[ords_prods['orders_day_of_week']==1]

,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,Busiest_day
33,550135,1,7,1,9,20.0,196,1,1,Soda,77,7,9.0,Mid-range product,Busiest days
34,550135,1,7,1,9,20.0,10258,2,1,Pistachios,117,19,3.0,Low-range product,Busiest days
35,550135,1,7,1,9,20.0,12427,3,1,Original Beef Jerky,23,19,4.4,Low-range product,Busiest days
36,550135,1,7,1,9,20.0,25133,4,1,Organic String Cheese,21,16,8.6,Mid-range product,Busiest days
37,550135,1,7,1,9,20.0,13032,5,1,Cinnamon Toast Crunch,121,14,4.0,Low-range product,Busiest days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32434207,2977660,206209,13,1,12,7.0,14197,5,1,Tomato Paste,9,9,5.6,Mid-range product,Busiest days
32434208,2977660,206209,13,1,12,7.0,38730,6,0,Brownie Crunch High Protein Bar,3,19,5.9,Mid-range product,Busiest days
32434209,2977660,206209,13,1,12,7.0,31477,7,0,High Protein Bar Chunky Peanut Butter,3,19,4.2,Low-range product,Busiest days
32434210,2977660,206209,13,1,12,7.0,6567,8,0,Chocolate Peanut Butter Protein Bar,3,19,4.9,Low-range product,Busiest days


In [346]:
ords_prods['Busiest_day'].value_counts(dropna=False)

Busiest_day
Regularly Busy    12925418
Busiest days      11873677
Slowest days       7629990
Name: count, dtype: int64

In [347]:
ords_prods['orders_day_of_week'].value_counts(dropna=False)

orders_day_of_week
0    6208779
1    5664898
6    4499643
2    4217041
5    4208734
3    3843427
4    3786563
Name: count, dtype: int64

# When acuracy checking, each part of the DF was filtered for 0,1,3,or4. With this results the "Busiest day" or "Slowest day" labels applied to the top and bottom 2 respective days. In addition, frequency counts were conducted and all of the values under 0,1 add up to the same value for a "busiest days". All of the values under 3,4 add up to the same value of a "slowest day". 

# 6208779 [0] + 5664898 [1] = 11873677 [Busiest days value]
# 3843427 [3] + 3786563[4] = 7629990 [Slowest Days]


## #4. When too many users make Instacart orders at the same time, the app freezes. The senior technical officer at Instacart wants you to identify the busiest hours of the day. Rather than by hour, they want periods of time labeled “Most orders,” “Average orders,” and “Fewest orders.” Create a new column containing these labels called “busiest_period_of_day.”

In [358]:
# Value counts will be found of the hour of day column
ords_prods['order_time_hour_of_day'].value_counts(dropna= False)

order_time_hour_of_day
10    2763963
11    2738139
14    2691155
15    2664096
13    2662888
12    2620372
16    2537011
9     2456300
17    2089071
8     1719697
18    1637646
19    1259153
20     976861
7      891797
21     796253
22     634671
23     402568
6      290727
0      218915
1      115769
5       88040
2       69416
4       53273
3       51304
Name: count, dtype: int64

In [369]:
#Using the frequency counts, will separate the values into 3 parts equally to define the 3 categories


#With this data, I will set the parameters as:
# Most orders = [10, 11, 14, 15, 13, 12, 16, 9]
# Average orders = [17, 8, 18, 19, 20, 7, 21, 22]
# Fewest orders = [23, 6, 0, 1, 5, 2, 4, 3]

# For consistency checks, there are 8 numbers in each category, so that adds up to the total 24 time segments
# A For statement will be used to write the new column under the busy_periods variable
# Goal is to use the for statement to look into the list and assign the respective value to the variable
busy_periods = []  
for value in ords_prods['order_time_hour_of_day']:  
    if value in [10, 11, 14, 15, 13, 12, 16, 9]:  
        busy_periods.append('Most orders')  
    elif value in [17, 8, 18, 19, 20, 7, 21, 22]:
        busy_periods.append('Average orders')
    elif value in [23, 6, 0, 1, 5, 2, 4, 3]:
        busy_periods.append('Fewest orders')
    else:
        busy_periods.append('Not enough data')

busy_periods

['Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Most orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Average orders',
 'Avera

In [378]:
#Applying this new variable to a column in the ords_prods dataframe

ords_prods['busiest_period_of_day']=busy_periods
ords_prods.head()

/var/folders/c3/gxzbvq9x1032lnrpvbyhdtrw0000gn/T/ipykernel_9272/3154718274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ords_prods['busiest_period_of_day']=busy_periods


,order_id,user_id,order_number,orders_day_of_week,order_time_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,price_range_loc,Busiest_day,busiest_period_of_day
0,2539329,1,1,2,8,11.114836,196,1,0,Soda,77,7,9.0,Mid-range product,Regularly Busy,Average orders
1,2539329,1,1,2,8,11.114836,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,Mid-range product,Regularly Busy,Average orders
2,2539329,1,1,2,8,11.114836,12427,3,0,Original Beef Jerky,23,19,4.4,Low-range product,Regularly Busy,Average orders
3,2539329,1,1,2,8,11.114836,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,Low-range product,Regularly Busy,Average orders
4,2539329,1,1,2,8,11.114836,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,Low-range product,Regularly Busy,Average orders


## 5. Print the frequency for this new column.

In [375]:
#This script prints the frequency values of the new column
ords_prods['busiest_period_of_day'].value_counts(dropna=False)

#There seems to be no errors when doing a consistency check on the new column

busiest_period_of_day
Most orders       21133924
Average orders    10005149
Fewest orders      1290012
Name: count, dtype: int64

## 6. Export your dataframe as a pickle file (since you added new columns) and store it correctly in your “Prepared Data” folder.

In [382]:
ords_prods.to_pickle(os.path.join(path, '02 Data','Prepared Data', '4.7 ords_prods_merged.pkl'))
